<a href="https://colab.research.google.com/github/dabin3178/Korean-Newords-NLP/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KoNLPy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 1.1 MB/s 
     |████████████████████████████████| 448 kB 68.6 MB/s 
     |████████████████████████████████| 86 kB 7.1 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import konlpy
import re
okt = konlpy.tag.Okt()

def text_preprocess(x): #한글 완성형만 걸러냄.
    text=[]
    a = re.sub('[^가-힣\\s]', '',x)
    for j in a.split():
        text.append(j)
    return ' '.join(text)

def dict_preprocess(x): #공백도 제외
    text=[]
    a = re.sub('[^가-힣]', '',x)
    for j in a.split():
        text.append(j)
    return ' '.join(text)

def tokenize(x): # 이거 안 써요.
    text = []
    tokens = okt.pos(x)
    for token in tokens :
        if token[1] != 'Noun' or len(token[0]) == 1 or len(token[0]) > 6:
            continue
        else:
            text.append(token[0])
    return text


def make_ngrams(x): # 부분 글자 만드는 함수
    result = []
    for i in range(1, len(x) + 1):
        ngram = []
        for j in range(0, len(x) - i + 1):
            ngram.append(x[j:j+i])
        result.extend(ngram)

    return result

def remove_stopwords(sentence, stopwords): # 이거 안 써요
  for word in sentence:
    if word in stopwords:
      sentence.remove(word)
  return sentence

In [ ]:
#무시하는 부분

import pandas as pd

eta_global_df = pd.read_csv("/content/drive/MyDrive/Project_FINAL/raw_data/Globalfree.csv")
eta_sisa_df = pd.read_csv("/content/drive/MyDrive/Project_FINAL/raw_data/df_에타_sisa.csv")
eta_secret_df = pd.read_csv("/content/drive/MyDrive/Project_FINAL/raw_data/eta_비밀.csv")
dc_inside_df = pd.read_csv("/content/drive/MyDrive/Project_FINAL/raw_data/df_DC_total.csv")
naver_news_df = pd.read_csv("/content/drive/MyDrive/Project_FINAL/raw_data/navernews.csv")
deoku_df = pd.read_csv("/content/drive/MyDrive/Project_FINAL/raw_data/deoku.csv")
nate_pann_df = pd.read_csv("/content/drive/MyDrive/Project_FINAL/raw_data/네이트판_통합.csv")
ppomppu1_df = pd.read_csv("/content/drive/MyDrive/Project_FINAL/raw_data/ppomppu1.csv")
ppomppu2_df = pd.read_csv("/content/drive/MyDrive/Project_FINAL/raw_data/ppomppu2.csv")
youtube_df = pd.read_csv("/content/drive/MyDrive/Project_FINAL/raw_data/유튜브15개.csv")

total_data_df = pd.concat([eta_global_df, eta_sisa_df],ignore_index=True)
total_data_df = pd.concat([total_data_df, eta_secret_df], ignore_index=True)
total_data_df = pd.concat([total_data_df, dc_inside_df], ignore_index=True)
total_data_df = pd.concat([total_data_df, naver_news_df], ignore_index=True)
total_data_df = pd.concat([total_data_df, deoku_df], ignore_index=True)
total_data_df = pd.concat([total_data_df, nate_pann_df], ignore_index=True)
total_data_df = pd.concat([total_data_df, ppomppu1_df], ignore_index=True)
total_data_df = pd.concat([total_data_df, ppomppu2_df], ignore_index=True)
total_data_df = pd.concat([total_data_df, youtube_df], ignore_index=True)

total_data_df = total_data_df.drop(columns=["제목", "내용", "title", "content", "vote", "Source.Name", "Column1", "date", "Unnamed: 0", "url", "ID"], axis=1)

total_data_df.to_csv("/content/drive/MyDrive/Project_FINAL/total_data.csv", index=False, encoding="utf-8-sig")

total_data_df

In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Project/total_data.csv", encoding='utf8', engine='python') # 데이터 불러오기

df['process'] = df['comment'].apply(lambda x : text_preprocess(x))  # 정규표현식으로 필요한 부분만 걸러냄.
df['process'] = df['process'].apply(lambda x: x.split()) # 어절 단위로 문장을 나눔.

words = [] # 여러 어절을 하나의 리스트로 저장할 배열
partial_words = [] # 여러 부분 단어를 하나의 리스트로 저장할 배열

# 여러 어절을 하나의 리스트로 저장
for i in range(len(df['process'])):
    words.extend(df['process'][i])

# 데이터프레임으로 만들기 위한 형태 변환
data = {
    'words' : list(set(words))
}

# 각 어절을 가지고 데이터 프레임을 만듦.
new_words_df = pd.DataFrame(data)

new_words_df['candidate'] = new_words_df['words'].apply(lambda x: make_ngrams(x)) # 어절에 해당하는 부분 글자를 구함.
new_words_df['phoneme_cnt'] = new_words_df['words'].apply(lambda x: len(x)) # 각 어절의 음절 수를 계산.

# 여러 부분 단어를 하나의 리스트로 저장
for i in range(len(new_words_df['candidate'])):
    partial_words.extend(new_words_df['candidate'][i])
    

word_segment_cnt = {k: 0 for k in range(1, 8)} # 각 어절의 길이에 따른 빈도 분포를 저장할 리스트
word_segment_ratio = {k: 0 for k in range(1, 8)} # 각 어절의 길이에 따른 빈도 분포 비율을 저장할 리스트

# 각 어절의 길이에 따른 분포를 계산함.
for i in range(len(new_words_df['phoneme_cnt'])):
    if new_words_df['phoneme_cnt'][i] >= 7:
        word_segment_cnt[7] += 1
    else:
        word_segment_cnt[new_words_df['phoneme_cnt'][i]] += 1

total_sum = new_words_df.shape[0]

# 각 어절별 빈도 분포 비율을 저장하는 과정
for k, v in word_segment_cnt.items():
    word_segment_ratio[k] = (v / total_sum) * 100

In [ ]:
new_words_df

,words,candidate,phoneme_cnt
0,팩트체크함,"[팩, 트, 체, 크, 함, 팩트, 트체, 체크, 크함, 팩트체, 트체크, 체크함,...",5
1,상향,"[상, 향, 상향]",2
2,싸웠다는거,"[싸, 웠, 다, 는, 거, 싸웠, 웠다, 다는, 는거, 싸웠다, 웠다는, 다는거,...",5
3,묘목기부,"[묘, 목, 기, 부, 묘목, 목기, 기부, 묘목기, 목기부, 묘목기부]",4
4,폭투,"[폭, 투, 폭투]",2
...,...,...,...
635495,순간이지만,"[순, 간, 이, 지, 만, 순간, 간이, 이지, 지만, 순간이, 간이지, 이지만,...",5
635496,혈압오를짤,"[혈, 압, 오, 를, 짤, 혈압, 압오, 오를, 를짤, 혈압오, 압오를, 오를짤,...",5
635497,학폭설,"[학, 폭, 설, 학폭, 폭설, 학폭설]",3
635498,영선좌가,"[영, 선, 좌, 가, 영선, 선좌, 좌가, 영선좌, 선좌가, 영선좌가]",4


In [ ]:
from tqdm import tqdm

freq = [0] * len(partial_words)

for i in tqdm(range(len(partial_words))):
    for j in range(len(words)):
        if partial_words[i] in words[j]:
            freq[i] += 1

freq

  0%|          | 1042/9070531 [10:08<1464:43:55,  1.72it/s]

In [ ]:
# 부분 단어를 데이터 프레임으로 변환하기 위한 형태 변환
partial_data = {
    'partial_words': partial_words
}

# 데이터 프레임 생성
partial_words_df = pd.DataFrame(partial_data)

freq_ratio = [] # 부분 단어의 빈도 비율을 저장할 리스트

total_freq = sum(list(freq))

# 부분 단어의 빈도 비율 계산
for i in range(len(freq)):
    freq_ratio.append((freq[i]/total_freq)*100)

# 새롭게 부분 단어와 빈도 관련 데이터 프레임 생성 위함.
partial_data = {
    'partial_words': partial_words.copy(),
    'freq': freq,
    'freq_ratio': freq_ratio
}

# 데이터 프레임 생성.
partial_words_df = pd.DataFrame(partial_data, index=[i for i in range(0, len(freq))])

partial_words_df

In [ ]:
total_freq, sum(partial_words_df['freq']) # 실제 전체 빈도수가 잘 계산 됐는지 확인.

In [ ]:
new_words_df.shape # 전체 어절의 개수

In [ ]:
word_segment_cnt, sum(word_segment_cnt.values()) # 총 어절의 음절 길이에 따른 빈도 수

In [ ]:
word_segment_ratio # 총 어절의 음절 길이에 따른 빈도 분포 비율

In [ ]:
# 불필요한 정보를 걸러 저장하기 위한 리스트들
partial_words_extracted = []
freq_extracted = []
freq_ratio_extracted = []

freq_criteria = 20 # 걸러내지 않을 최소 등장 빈도 수

# 부분 단어의 길이가 1 혹은 7 이상, 아니면 그 부분 단어의 등장 빈도수가 기준 빈도수보다 낮으면 걸러냄.
for i in range(len(partial_words_df['freq'])):
    if partial_words_df['freq'][i] >= freq_criteria and (len(partial_words_df['partial_words'][i]) > 1  and len(partial_words_df['partial_words'][i]) < 7) :
        partial_words_extracted.append(partial_words_df['partial_words'][i])
        freq_extracted.append(partial_words_df['freq'][i])
        freq_ratio_extracted.append(partial_words_df['freq_ratio'][i])

# 데이터 프레임 생성을 위한 딕셔너리
partial_data = {
    'partial_words': partial_words_extracted,
    'freq': freq_extracted,
    'freq_ratio': freq_ratio_extracted
}
# 데이터 프레임 생성
partial_words_df = pd.DataFrame(partial_data)

partial_words_df.shape

In [ ]:
partial_words_df

In [ ]:
# 국어사전에서 단어를 불러오고 확인.
official_dict = pd.read_csv("/content/drive/MyDrive/Project_FINAL/dict/우리말샘_어활검품2.csv")
official_dict = list(official_dict['어휘'])

for i in range(len(official_dict)):
    official_dict[i] = dict_preprocess(official_dict[i])

tmp = []
for i in range(len(official_dict)):
    if official_dict[i] != '':
        tmp.append(official_dict[i])

official_dict = tmp

In [ ]:
official_data = {
    "어휘": official_dict

}

official_dict_df = pd.DataFrame(official_data)
official_dict_df.to_csv("/content/drive/MyDrive/Project_FINAL/dict/우리말샘_전처리.csv", encoding="utf-8-sig",index=False)

In [ ]:
# 국어사전과 비교하여 사전에 없는 단어만 뽑아내어 저장할 리스트
partial_words_extracted = []
freq_extracted = []
freq_ratio_extracted = []

# 문자열 인덱싱을 위한 사전의 딕셔너리 형태 변환.
official_dict = {official_dict[i]: "" for i in range(len(official_dict))}

for i in range(len(partial_words_df['partial_words'])):
    try:
        official_dict[partial_words_df['partial_words'][i]] # 사전에 있는지 확인.
    except KeyError: # 없으면 저장.
        partial_words_extracted.append(partial_words_df['partial_words'][i])
        freq_extracted.append(partial_words_df['freq'][i])
        freq_ratio_extracted.append(partial_words_df['freq_ratio'][i])

# 데이터 프레임 변환을 위한 딕셔너리
partial_data = {
    'partial_words': partial_words_extracted,
    'freq': freq_extracted,
    'freq_ratio': freq_ratio_extracted,
}
# 데이터 프레임 생성
partial_words_df = pd.DataFrame(partial_data)

partial_words_df.shape

(50462, 3)

In [ ]:
new_words_df.to_csv("/content/drive/MyDrive/Project_FINAL/ngrams.csv", encoding='utf-8-sig', index=False) # 어절에 관한 정보

In [ ]:
partial_words_df.to_csv("/content/drive/MyDrive/Project_FINAL/partial_total.csv", encoding='utf-8-sig', index=False) # 부분 단어에 대한 정보

In [ ]:
df.to_csv("/content/drive/MyDrive/Project_FINAL/origin_preprocessed.csv", encoding='utf-8-sig', index=False) # 원본 문장에 대한 전처리에 대한 정보

In [ ]:
adf = pd.read_csv("/content/drive/MyDrive/Project_FINAL/partial_total.csv")
adf

,partial_words,freq,freq_ratio
0,하는,42350,0.162095
1,는데,30803,0.117899
2,진짜,22045,0.084377
3,는거,18113,0.069328
4,림픽,13087,0.050091
...,...,...,...
50457,스터라,20,0.000077
50458,산다고,20,0.000077
50459,본인을,20,0.000077
50460,답일,20,0.000077
